In [8]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tasks import RoadTypeClfTask, TravelTimeEstimation
from sklearn import metrics, linear_model
from evaluation import Evaluation
from generator import RoadNetwork
import torch
from models import GAEModel
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
network = RoadNetwork()
network.load("../osm_data/porto")

In [10]:
decoder = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])
roadclf = RoadTypeClfTask(decoder, y)
roadclf.register_metric(name="f1_micro", metric_func=metrics.f1_score, args={"average": "micro"})
roadclf.register_metric(name="f1_macro", metric_func=metrics.f1_score, args={"average": "macro"})
roadclf.register_metric(name="f1_weighted", metric_func=metrics.f1_score, args={"average": "weighted"})

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gae = GAEModel(1,  128, device=device) # pseudo init
gae.load_emb(path="../model_states/gae/embedding.out")

array([[ 0.01818256,  0.07382999,  0.03331254, ..., -0.13202006,
         0.33308628,  0.36450168],
       [ 0.03449438,  0.08260331,  0.11854216, ..., -0.11856375,
         0.4422397 ,  0.49484941],
       [ 0.14319448,  0.24921644,  0.11281582, ..., -0.07663582,
         0.36288461,  0.3798165 ],
       ...,
       [-0.00205083, -0.12101499,  0.04077854, ...,  0.19816038,
         0.06837461,  0.05633661],
       [-0.03523884,  0.0175008 ,  0.23978847, ..., -0.15987927,
        -0.10179204, -0.1786136 ],
       [-0.03523884,  0.0175008 ,  0.23978847, ..., -0.15987927,
        -0.10179204, -0.1786136 ]])

In [21]:
eva = Evaluation()
eva.register_task("Road Type CLF", roadclf)
eva.register_model("GAE", gae)
res = eva.run()

In [22]:
print(res[0][0])
res[0][1].head()

Road Type CLF


,f1_micro,f1_macro,f1_weighted
GAE,0.715042,0.379847,0.694873
